In [2]:
import numpy as np
import pandas as pd
from astropy.io import fits
from astropy.table import Table
import glob

import matplotlib.pyplot as plt
%matplotlib notebook

In [44]:
gaia_dir = "/home/xhall/Documents/GaiaCross/"
gaia_files = glob.glob("/home/xhall/Documents/GaiaCross/Reprinted/*.csv")
for file_num, gf in enumerate(gaia_files):
    gdat = pd.read_csv(gf)
    
    pm = np.sqrt(gdat["pmra"]**2 + gdat["pmdec"]**2)
    pm_unc = np.sqrt(gdat["pmra"] ** 2 * gdat["pmra_error"] ** 2 /(gdat["pmra"] ** 2 + gdat["pmdec"] ** 2) +
            gdat["pmdec"] ** 2 * gdat["pmdec_error"] ** 2 / (gdat["pmra"] ** 2 + gdat["pmdec"] ** 2) +
            2 * gdat["pmra"] * gdat["pmdec"]/(gdat["pmra"] + gdat["pmdec"] ** 2) * gdat["pmra_pmdec_corr"]*gdat["pmra_error"]*gdat["pmdec_error"])
    gdat["pm_unc"] = pm_unc
    gdat["pm"] = pm
    
    gdat = Table.from_pandas(gdat)
    break

In [45]:
gdat_cut = gdat[np.where((gdat["phot_bp_rp_excess_factor"] < 1.3+0.06*np.power(gdat["phot_bp_mean_mag"]-gdat["phot_rp_mean_mag"],2)) &
        (gdat["phot_bp_rp_excess_factor"] > 1.0+0.015*np.power(gdat["phot_bp_mean_mag"]-gdat["phot_rp_mean_mag"],2)) &
        (gdat["astrometric_chi2_al"]/(gdat["astrometric_n_good_obs_al"]-5) < 1.44*np.maximum(1,np.exp(-0.4*(gdat["phot_g_mean_mag"]-19.5))))
        )]

The following query was used to select **obvious stars** from the Gaia-PS1 crossmatch table: 

    select ps1.*,
    sqrt(power(pmra,2) + power(pmdec,2)) as pm, 
    sqrt(  power(pmra,2)*power(pmra_error, 2)/(power(pmra,2) + power(pmdec,2)) 
	 + power(pmdec,2)*power(pmdec_error, 2)/(power(pmra,2) + power(pmdec,2)) 
	 + 2*pmra*pmdec/(power(pmra,2) + power(pmdec,2))*pmra_pmdec_corr*pmra_error*pmdec_error) as pm_unc, 
    gaia.parallax_over_error, gaia.phot_g_mean_mag, gaia.b
    from gaiadr2.panstarrs1_best_neighbour as ps1 
    inner join gaiadr2.gaia_source as gaia
    on ps1.source_id = gaia.source_id
    where gaia.astrometric_params_solved > 3
    and gaia.dec between -10 and -5
    and phot_bp_rp_excess_factor < 1.3+0.06*power(phot_bp_mean_mag-phot_rp_mean_mag,2)
    AND phot_bp_rp_excess_factor > 1.0+0.015*power(phot_bp_mean_mag-phot_rp_mean_mag,2)
    AND astrometric_chi2_al/(astrometric_n_good_obs_al-5) < 1.44*greatest(1,exp(-0.4*(phot_g_mean_mag-19.5)))
    order by ps1.original_ext_source_id

where the above query produced the file `dec_neg10-result.fits` and the declination range was adjusted to create all the other files.

## Create the loop to put all the objid into a single file

In [36]:
gaia_dir = "/home/xhall/Documents/GaiaCross/"
gaia_files = glob.glob("/home/xhall/Documents/GaiaCross/Reprinted/*.csv")
for file_num, gf in enumerate(gaia_files):
    gdat = pd.read_csv(gf)
    
    pm = np.sqrt(gdat["pmra"]**2 + gdat["pmdec"]**2)
    pm_unc = np.sqrt(gdat["pmra"] ** 2 * gdat["pmra_error"] ** 2 /(gdat["pmra"] ** 2 + gdat["pmdec"] ** 2) +
            gdat["pmdec"] ** 2 * gdat["pmdec_error"] ** 2 / (gdat["pmra"] ** 2 + gdat["pmdec"] ** 2) +
            2 * gdat["pmra"] * gdat["pmdec"]/(gdat["pmra"] + gdat["pmdec"] ** 2) * gdat["pmra_pmdec_corr"]*gdat["pmra_error"]*gdat["pmdec_error"])
    gdat["pm_unc"] = pm_unc
    gdat["pm"] = pm
    
    gdat = Table.from_pandas(gdat)
    
    gdat = gdat[np.where((gdat["phot_bp_rp_excess_factor"] < 1.3+0.06*np.power(gdat["phot_bp_mean_mag"]-gdat["phot_rp_mean_mag"],2)) &
        (gdat["phot_bp_rp_excess_factor"] > 1.0+0.015*np.power(gdat["phot_bp_mean_mag"]-gdat["phot_rp_mean_mag"],2)) &
        (gdat["astrometric_chi2_al"]/(gdat["astrometric_n_good_obs_al"]-5) < 1.44*np.maximum(1,np.exp(-0.4*(gdat["phot_g_mean_mag"]-19.5))))
        )]
    
    parallax_stars = np.where( (gdat['parallax_over_error'] >= 8) & 
                               (gdat['number_of_neighbours'] == 1) & 
                               (gdat['number_of_mates'] == 0))
    pm_stars = np.where((gdat['pm']/gdat['pm_unc'] >= 7.5) & 
                         (gdat['number_of_neighbours'] == 1) & 
                         (gdat['number_of_mates'] == 0))
    tmp_df = pd.DataFrame(gdat['original_ext_source_id'][parallax_stars]).rename(columns={"original_ext_source_id": "objid"})
    big_df = tmp_df.copy()
    big_df['pm_over_error'] = gdat['pm'][parallax_stars]/gdat['pm_unc'][parallax_stars]
    big_df['parallax_over_error'] = gdat['parallax_over_error'][parallax_stars]
    big_df['g_mag'] = gdat['phot_g_mean_mag'][parallax_stars]
    
    print(file_num, gf.split("/")[-1], len(gdat[parallax_stars]), 
          len(parallax_stars[0])/len(gdat))
    if file_num == 0:
        master_df = tmp_df.copy()
        master_big = big_df.copy()
    else:
        master_df = master_df.append(tmp_df, ignore_index=True)
        master_big = master_big.append(big_df, ignore_index=True)

/home/xhall/anaconda3/lib/python3.7/site-packages/pandas/core/series.py:679: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/xhall/anaconda3/lib/python3.7/site-packages/astropy/table/column.py:1020: RuntimeWarning: invalid value encountered in less
  result = getattr(super(), op)(other)
/home/xhall/anaconda3/lib/python3.7/site-packages/astropy/table/column.py:1020: RuntimeWarning: invalid value encountered in greater
  result = getattr(super(), op)(other)
/home/xhall/anaconda3/lib/python3.7/site-packages/astropy/table/column.py:1020: RuntimeWarning: invalid value encountered in greater_equal
  result = getattr(super(), op)(other)


0 Cross4.csv 367 0.000343076904380634
1 Cross6.csv 0 0.0
2 Cross8.csv 0 0.0
3 Cross5.csv 0 0.0
4 Cross3.csv 0 0.0
5 Cross7.csv 1 3.822995316830737e-06
6 Cross12.csv 3174 0.00289448006055245
7 Cross2.csv 136 0.0003556438854094481
8 Cross10.csv 10 0.0001758860258552458
9 Cross9.csv 0 0.0
10 Cross11.csv 0 0.0
11 Cross1.csv 726 0.0006541253596112729
12 Cross0.csv 0 0.0


In [37]:
master_df.to_hdf(gaia_dir + "parallax_objid_stars.h5", "d1")
len(master_df)

4414

In [38]:
master_big.to_hdf(gaia_dir + "parallax_ps1_gaia_mag_pm_plx.h5", "d1")
len(master_big)

4414

In [39]:
for file_num, gf in enumerate(gaia_files):
    gdat = pd.read_csv(gf)
    
    pm = np.sqrt(gdat["pmra"]**2 + gdat["pmdec"]**2)
    pm_unc = np.sqrt(gdat["pmra"] ** 2 * gdat["pmra_error"] ** 2 /(gdat["pmra"] ** 2 + gdat["pmdec"] ** 2) +
            gdat["pmdec"] ** 2 * gdat["pmdec_error"] ** 2 / (gdat["pmra"] ** 2 + gdat["pmdec"] ** 2) +
            2 * gdat["pmra"] * gdat["pmdec"]/(gdat["pmra"] + gdat["pmdec"] ** 2) * gdat["pmra_pmdec_corr"]*gdat["pmra_error"]*gdat["pmdec_error"])
    gdat["pm_unc"] = pm_unc
    gdat["pm"] = pm
    
    gdat = Table.from_pandas(gdat)
    
    gdat = gdat[np.where((gdat["phot_bp_rp_excess_factor"] < 1.3+0.06*np.power(gdat["phot_bp_mean_mag"]-gdat["phot_rp_mean_mag"],2)) &
        (gdat["phot_bp_rp_excess_factor"] > 1.0+0.015*np.power(gdat["phot_bp_mean_mag"]-gdat["phot_rp_mean_mag"],2)) &
        (gdat["astrometric_chi2_al"]/(gdat["astrometric_n_good_obs_al"]-5) < 1.44*np.maximum(1,np.exp(-0.4*(gdat["phot_g_mean_mag"]-19.5))))
        )]
    
    parallax_stars = np.where( (gdat['parallax_over_error'] >= 8) & 
                               (gdat['number_of_neighbours'] == 1) & 
                               (gdat['number_of_mates'] == 0))
    pm_stars = np.where((gdat['pm']/gdat['pm_unc'] >= 7.5) &
                         (gdat['number_of_neighbours'] == 1) & 
                         (gdat['number_of_mates'] == 0))
    tmp_df = pd.DataFrame(gdat['original_ext_source_id'][pm_stars]).rename(columns={"original_ext_source_id": "objid"})
    big_df = tmp_df.copy()
    big_df['pm_over_error'] = gdat['pm'][pm_stars]/gdat['pm_unc'][pm_stars]
    big_df['parallax_over_error'] = gdat['parallax_over_error'][pm_stars]
    big_df['g_mag'] = gdat['phot_g_mean_mag'][pm_stars]
    
    print(file_num, gf.split("/")[-1], len(gdat[pm_stars]), 
          len(pm_stars[0])/len(gdat))
    if file_num == 0:
        master_df = tmp_df.copy()
        master_big = big_df.copy()
    else:
        master_df = master_df.append(tmp_df, ignore_index=True)
        master_big = master_big.append(big_df, ignore_index=True)

0 Cross4.csv 26463 0.024737994879086424
1 Cross6.csv 1107 0.012424521313609733
2 Cross8.csv 2562 0.005940111985717763
3 Cross5.csv 5079 0.005619233067067909
4 Cross3.csv 992 0.012231058504407866
5 Cross7.csv 1891 0.007229284144126924
6 Cross12.csv 52677 0.048037972952023125
7 Cross2.csv 8521 0.022282658438043435
8 Cross10.csv 2271 0.03994371647172632
9 Cross9.csv 3626 0.009707802117189718
10 Cross11.csv 1221 0.022737853591314548
11 Cross1.csv 50240 0.045266195684394425
12 Cross0.csv 526 0.009644828281717


In [40]:
master_df.to_hdf(gaia_dir + "pm_objid_stars.h5", "d1")
len(master_df)

157176

In [41]:
master_big.to_hdf(gaia_dir + "pm_ps1_gaia_mag_pm_plx.h5", "d1")
len(master_big)

157176